In [1]:
import numpy as np
import pandas as pd
from boruta import BorutaPy
from sklearn.datasets import load_diabetes
from sklearn.metrics import accuracy_score 
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
%cd Z:\PPMI_Data\Excels\NonMotors\Derived250\Female\GeneCpG\STAIS
data = pd.read_csv('STAIS_with_MethylomeDetails_Female.csv')
data.head(1)

Z:\PPMI_Data\Excels\NonMotors\Derived250\Female\GeneCpG\STAIS


,Sentrix,PATNO,Gender,APPRDX,HYS,Age,AgeCate,STAIS_Category,cg26354221,cg02237342,...,cg12383788,cg09938164,cg16043968,cg06398244,cg26924480,cg06985259,cg09180977,cg05600023,cg01781529,cg02839875
0,200973410159_R03C01,3000,2,2,0,69.1,4,1,0.918085,0.016498,...,0.920545,0.820139,0.943315,0.920632,0.876711,0.929227,0.946104,0.848444,0.605773,0.629179


In [3]:
X = data.drop(['Sentrix','PATNO','HYS','APPRDX','Age','AgeCate','Gender','STAIS_Category'], axis = 1)
y = data['APPRDX']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1) 

In [4]:
rf_all_features = RandomForestClassifier(random_state=1)
rf_all_features.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [5]:
accuracy_score(y_test, rf_all_features.predict(X_test))

0.8333333333333334

In [6]:
rfc = RandomForestClassifier(random_state=1)
boruta_selector = BorutaPy(rfc, n_estimators= 'auto', verbose=1, random_state=1)
boruta_selector.fit(np.array(X_train), np.array(y_train))  

Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100
Iteration: 9 / 100
Iteration: 10 / 100
Iteration: 11 / 100
Iteration: 12 / 100
Iteration: 13 / 100
Iteration: 14 / 100
Iteration: 15 / 100
Iteration: 16 / 100
Iteration: 17 / 100
Iteration: 18 / 100
Iteration: 19 / 100
Iteration: 20 / 100
Iteration: 21 / 100
Iteration: 22 / 100
Iteration: 23 / 100
Iteration: 24 / 100
Iteration: 25 / 100
Iteration: 26 / 100
Iteration: 27 / 100
Iteration: 28 / 100
Iteration: 29 / 100
Iteration: 30 / 100
Iteration: 31 / 100
Iteration: 32 / 100
Iteration: 33 / 100
Iteration: 34 / 100


BorutaPy finished running.

Iteration: 	35 / 100
Confirmed: 	3
Tentative: 	0
Rejected: 	3039


BorutaPy(estimator=RandomForestClassifier(n_estimators=24,
                                          random_state=RandomState(MT19937) at 0x213A6F8A740),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x213A6F8A740, verbose=1)

In [7]:
# print("Ranking: ",boruta_selector.ranking_)          
print("No. of significant features: ", boruta_selector.n_features_)

No. of significant features:  3


In [8]:
selected_rf_features = pd.DataFrame({'CpG':list(X_train.columns),
                                       'Ranking':boruta_selector.ranking_})
selected_rf_features = selected_rf_features.sort_values(by='Ranking')

In [9]:
Top50 = selected_rf_features.iloc[:50, :]
Top50.head(3)

,CpG,Ranking
949,cg01600124,1
2094,cg08271803,1
1970,cg26413900,1


In [10]:
Top50.to_csv ('STAIS_BorutaFemale_Top50.csv', index = False)
selected_rf_features.to_csv ('STAIS_BorutaFemale_All.csv', index = False)

#Identify a string in dataframe
substring = 'cg15328505'
selected_rf_features[selected_rf_features.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
